##**Proyecto Etapa 1 - Construcción de modelos de analítica de textos**

**Caso  Fondo de Poblaciones de las Naciones Unidas (UNFPA)**

**Integrantes**

Mateo Calderon

Juan Ramirez

Daniela Castrillón


#**Entendimiento y preparación de los datos**

A continuación, se describen las características más relevantes de los datos con el fin de realizar un perfilamiento de los mismos; adicional, se realiza el análisis de calidad de datos.

Por otro lado, se incluye el tratamiento de los datos (preparación o transformaciones requeridas), de acuerdo con el dominio, las técnicas y los algoritmos seleccionados para resolver el problema planteado.


##Carga de datos

En esta sección, se cargan los datos necesarios para el análisis y se importan las librerías necesarias para leer el archivo que contiene los datos de entrenamiento y prueba.

Es esencial comenzar con este paso para asegurarnos de que todos los datos necesarios están correctamente importados para las etapas siguientes del análisis.

In [ ]:
!pip install unidecode
!pip install shap

In [ ]:
import pandas as pd
from unidecode import unidecode
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from IPython.display import display
from sklearn.svm import SVC
import shap

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.inspection import permutation_importance

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, ConfusionMatrixDisplay


In [ ]:
#Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Ruta del archivo Excel
train_data = '/content/sample_data/ODScat_345.xlsx'
test_data = '/content/sample_data/TestODScat_345.xlsx'

#Cargar el archivo Excel
data = pd.read_excel(train_data)
test_data = pd.read_excel(test_data)

#Visualizar los datos para corroborar que hayan quedado bien
pd.set_option('display.max_colwidth', None)
display(data)


,Textos_espanol,sdg
0,"Por ejemplo, el nÃºmero de consultas externas de especialistas es de 319 por cada mil derechohabientes en el SP, en comparaciÃ³n con 338 y 620 por cada mil derechohabientes en el IMSS y el ISSSTE, respectivamente. Si bien algunas de estas diferencias pueden reflejar una necesidad desigual (como la poblaciÃ³n ligeramente mayor del ISSSTE), otras no pueden justificarse de esta manera. El nÃºmero de recetas que no pudieron ser surtidas en su totalidad por un farmacÃ©utico debido a la falta de existencias es de 33% dentro del SP en comparaciÃ³n con 14% dentro del IMSS segÃºn los datos de la encuesta (aunque las propias cifras de los institutos de la SS sugieren tasas mÃ¡s altas de recetas surtidas). Ambas cifras se encuentran entre las mÃ¡s altas de la OCDE. El gasto de bolsillo no se ha reducido significativamente en la última década, a pesar de los esfuerzos para lograr la cobertura sanitaria universal a través de la reforma del SP.",3
1,"En 2007, el gobierno central financió directamente solo el 0,3% del gasto total en salud, pero realizó transferencias específicas para el gasto en salud que ascendieron a otro 5,6% del gasto total. La mayor parte del gasto presupuestario en salud lo realizan los gobiernos de los condados. Por lo tanto, los condados pobres solo pueden ofrecer un bajo nivel de atención a la población local. El gobierno provincial proporciona subsidios del lado de la oferta a los hospitales.",3
2,"Claramente, hay muchos otros factores en juego, en particular, una variedad de barreras y determinantes sociales. Estos pueden estar relacionados con el género, la clase, la etnia, la casta, el idioma y la religión, y surgen de estructuras, normas y procesos sociales arraigados que aceptan e incluso fomentan distribuciones injustas de la riqueza y los recursos sociales. Por ejemplo, las personas que viven con el VIH y algunas otras enfermedades crónicas enfrentan altos niveles de estigma y discriminación que dificultan su acceso a pruebas médicas, tratamiento, atención y apoyo. Por lo tanto, los gobiernos que buscan mejorar los estándares de salud querrán mejorar la condición de la mujer, brindar servicios de salud reproductiva efectivos, implementar programas integrales de lactancia materna en línea con las recomendaciones actuales y aumentar las presiones comunitarias contra el aborto selectivo por sexo. Las personas pobres, que carecen de educación e información, o de fondos para opciones más saludables, son más vulnerables al impacto de las ENT que los ricos y es probable que mueran antes. Directa e indirectamente, las ENT tendrán impactos de gran alcance en el progreso hacia los ODM.",3
3,"Por ejemplo, el estado australiano de Victoria creó una agencia de promoción de la salud financiada con los impuestos sobre el tabaco, y varios estados de EE.UU. (California, Massachusetts, Arizona y Oregón) han destinado parte de los ingresos procedentes de los impuestos especiales sobre el tabaco a actividades de educación y control del tabaco, mientras que en Nueva Jersey, Ohio y Texas los ingresos procedentes del alcohol se utilizan para programas de educación y prevención relacionados con el alcohol (Sassi et al., 2004), Los impuestos locales pueden ser una fuente importante de ingresos, sobre todo en los sistemas sanitarios descentralizados (por ejemplo, Dinamarca, Finlandia, Italia y Suecia)17 . Además, el nivel de los impuestos locales puede fijarse en función de las preferencias locales sobre el nivel y la calidad de los servicios médicos. Sin embargo, en ausencia de un mecanismo de transferencia ajustado al riesgo y gestionado centralmente, la descentralización reduce la mancomunación de riesgos y podría dar lugar a mayores disparidades sanitarias interregionales.",3
4,"El consumo anual de alcohol se estima en 15,7 litros al año para los hombres, más de un 50% por encima de la media de la OCDE (10,4) (Sistema mundial de información sobre alcohol y salud de la OMS), lo que puede ayudar a explicar la

##Exploración inicial de los datos

Luego de cargar los datos, se realizó una exploración inicial para obtener un entendimiento general del conjunto de datos. Esto incluye revisar las dimensiones, tipos de datos de las columnas y una muestra de los mismos para tener una idea clara de cómo están estructurados.

In [ ]:
#Dimensiones del conjunto de datos
data.shape

(4049, 2)

Teniendo en cuenta lo anterior, se determina que el dataframe cuenta con 4049 filas y 2 columnas.

Cada fila representa una opinión ciudadana relacionadas al tema de las ODS junto con un número que representa el ODS específico al que el texto está asociado.

Entender esto será útil para analizar la efectividad de clasificación de las opiniones, con el fin de desarrollar modelos que puedan predecir la categorización de la opinión dentro de las ODS específicas.

In [ ]:
#Visualizar los tipos de datos de las columnas
data.dtypes

,0
Textos_espanol,object
sdg,int64


La información anterior, permite planificar la limpieza y el preprocesamiento de los datos adecuadamente.

Se cuentan con una variable de tipo object lo cual indica que contiene texto y, con una variable númerica de tipo int.

In [ ]:
#Visualizar una muestra aleatoria del conjunto de datos
data.sample(5)

,Textos_espanol,sdg
660,"Para reforzar la garantía de la calidad de la asistencia, la atención no debe centrarse únicamente en el número de profesionales, sino también en sus perfiles profesionales y en la calidad de su actuación en la práctica. Se han tomado varias medidas para reforzar la posición de los ciudadanos/pacientes, centrándose en la tramitación de las reclamaciones. Sería aconsejable que Turquía dispusiera de más información de dominio público sobre el funcionamiento de los servicios sanitarios, y que la recogida sistemática de las experiencias de los usuarios de los servicios sanitarios estuviera más extendida. Cabe esperar que se siga trabajando en registros específicos. Es prioritario optimizar el uso de los datos administrativos, prestando especial atención a la mejora del intercambio de datos entre el Ministerio de Sanidad y el SSI. Es aconsejable una política coherente sobre cómo reforzar la infraestructura de información turca para facilitar el uso de indicadores de calidad, abordando temas como la vinculación de datos, el uso secundario de los datos de los historiales médicos electrónicos y la garantía de la privacidad y la seguridad de los datos.",3
2851,"La viudedad puede acarrear dificultades económicas y vulnerabilidad para las mujeres. Las condiciones de los derechos también están relacionadas con normas más específicas de las pensiones: por ejemplo, cuántos años de cotización se requieren (el llamado ""periodo de inversión""), el umbral de ingresos o activos para las prestaciones sujetas a la comprobación de recursos, y si las prestaciones se basan en los derechos o dependen de los recursos disponibles y las listas de espera. Las condiciones del derecho definen quién puede obtener una prestación y cómo se distribuye finalmente la cobertura.",5
3573,"Este capítulo explora cómo México puede enfrentar esos desafíos y los pasos que puede tomar para cerrar las brechas de género restantes en los principales puestos públicos de toma de decisiones. Mientras que las mujeres representaban solo el 22,6% de los miembros del parlamento en la Cámara de Diputados (Cámara de Diputados) en 2005, actualmente ocupan el 42,37% de los 500 escaños. En la Cámara de Senadores (Senado) ocupan el 33,6% de los 128 escaños, en comparación con el 17,19% en 2006. Dicha representación está muy por encima del promedio de la OCDE de 28,47% de escaños en cámaras bajas y legislaturas unicamerales.",5
3708,"Yu (2014) utiliza datos de los Estudios de Panel de Familias de China (CFPS) de 2010 y encuentra que el tiempo en el mercado laboral de las mujeres casadas y el nivel de ingresos están asociados negativamente con el tiempo que dedican a las tareas domésticas. Sus hallazgos indican un fenómeno llamado ""exhibición de género"" por el cual las esposas no logran reducir su tiempo de trabajo doméstico incluso cuando sus ingresos relativos aumentan hasta el punto de que ganan más que sus maridos. Las encuestas de uso del tiempo (TUS) han sido invaluables para estimar el tiempo que aportan los miembros del hogar y para medir todas las formas de trabajo.",5
3655,"La protection economique des femmes en age avance, depende aujourd'hui de plusieurs facteurs lie,y comprende des regies des systemes de retraite, des condition du marche du travail et des dispositions fa-miliales prises a diversos momentos. Cet artículo se penche sur les regies des systemes de retraite et leur interacción avec d'autres conditions sociales et du marche du travail au cours de la vie d'une femme, pour reproduire ou attenuer les inegalites entre les sexes au moment de la vieillesse. Une rubrique separee est consacree aux systemes de retraite sans cotisation, a leur evolution dans le monde et au potentiel qu'ils ont de resoudre les lacunes existantes en matiere d'acces a la protection au moment de la vieillesse, independamment du sexe et du niveau de revenu.",5


##Perfilamiento de los datos

A continuación, se presentan estadísticas descriptivas que resumen la tendencia central, dispersión y forma de la distribución de los datos. Dicha información es crucial para entender las características de cada variable y detectar anomalías como valores atípicos.

In [ ]:
#Estadísticas descriptivas de los datos
data.describe()

,sdg
count,4049.000000
mean,4.051124
std,0.814338
min,3.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


##Análisis de calidad de datos

Finalmente, se evalúa la calidad de los datos revisando la completitud, unicidad, consistencia y validez de los mismos. Esto permite identificar registros duplicados, valores faltantes y otros problemas que pueden afectar el análisis o rendimiento del modelo.


###Completitud
Evaluar la completitud implica identificar los valores faltantes de los datos, con el fin de tomar decisión en la etapa de limpieza y preparación de los datos.

In [ ]:
#Mostrar la cantidad de valores nulos por columna
data.isnull().sum()

,0
Textos_espanol,0
sdg,0


In [ ]:
#Porcentaje de valores faltantes por columna
data.isnull().mean() * 100

,0
Textos_espanol,0.0
sdg,0.0


Se puede observar que ninguna de las dos columnas del conjunto de datos cuenta con problemas de datos faltantes.

### Unicidad

La unicidad implica asegurarse de que no hayan duplicados totales en los registros, ya que esto puede distorsionar el análisis.

In [ ]:
#Verificar duplicados
data.duplicated().sum()

0

Como se muestra, no hay registros duplicados dentro del conjunto de datos. Esto quiere decir que no hay datos que sean copias exactas de otros registros.

###Consistencia

Evaluar la consistencia de los datos hace referencia a que los datos no contengan contradicciones ni errores de lógica.

Se puede identificar que hay una consistencia estructural en los datos, en donde cada columna corresponde al tipo de dato que debe ser. Por un lado, la columna 'Textos_espanol' debe corresponder a texto y la columna 'sdg' debe ser un valor numérico entero, tal y como se puede evidenciar en los datos mostrados anteriormente.

###Validez

Finalmente, revisar la validez de los datos asegura que los datos cumplan con los formatos y rangos esperados dentro de lo definido por el negocio.

In [ ]:
data.dtypes

,0
Textos_espanol,object
sdg,int64


In [ ]:
data.shape

(4049, 2)

Se observa que la entrada de los datos es válida y los tipos de variables son los adecuados para todos los datos.

##Limpieza y preparación de los datos

En este punto, se debe realizar una limpieza de los datos; sin embargo, como se evidenció anteriormente, no se cuenta con datos nulos, duplicados, invalidos e inconsistentes.

A pesar de lo anterior, se debe preparar y transformar un poco más los datos antes de introducirlos a los modelos para obtener los resultados esperados.

Este proceso de preparación consta de 4 puntos, en donde se filtrarán los datos paso a paso.

**Normalización del texto**

Convertir el texto completo a minúsculas, eliminar la puntuación y los caracteres y, realizar correcciones ortográficas si es necesario.

**Eliminación de stopwords**

Quitar las palabras comunes que no aportan significado relevante al análisis (como 'y', 'en', 'un', etc.)

**Tokenización**

Separar el texto en unidades básicas (tokens), generalmente palabras o frases significativas.

**Lematización o Stemming**

Reducir las palabras a su raíz para disminuir la variabilidad de las palabras manteniendo un significado.


In [ ]:
#Configurar NLTK Stopwords
stop_words = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')

def limpiar_texto(texto):
  #Convertir el texto en minúsculas
  texto = texto.lower()
  #Tokenizar el texto
  tokens = word_tokenize(texto, language = 'spanish')
  #Eliminar stopwords y palabras no alfabéticas, y aplicar stemming
  palabras_limpias = [stemmer.stem(palabra) for palabra in tokens if palabra.isalpha() and palabra not in stop_words]
  #Unir de nuevo las palabras en una cadena
  texto_limpio = ' '.join(palabras_limpias)
  return texto_limpio

#Aplicar la función de limpieza a la columna de opiniones
data['Textos_limpio'] = data['Textos_espanol'].apply(limpiar_texto)
#test_data['Textos_limpio'] = test_data['Textos_espanol'].apply(limpiar_texto)

data.head(3)



,Textos_espanol,sdg,Textos_limpio
0,"Por ejemplo, el nÃºmero de consultas externas de especialistas es de 319 por cada mil derechohabientes en el SP, en comparaciÃ³n con 338 y 620 por cada mil derechohabientes en el IMSS y el ISSSTE, respectivamente. Si bien algunas de estas diferencias pueden reflejar una necesidad desigual (como la poblaciÃ³n ligeramente mayor del ISSSTE), otras no pueden justificarse de esta manera. El nÃºmero de recetas que no pudieron ser surtidas en su totalidad por un farmacÃ©utico debido a la falta de existencias es de 33% dentro del SP en comparaciÃ³n con 14% dentro del IMSS segÃºn los datos de la encuesta (aunque las propias cifras de los institutos de la SS sugieren tasas mÃ¡s altas de recetas surtidas). Ambas cifras se encuentran entre las mÃ¡s altas de la OCDE. El gasto de bolsillo no se ha reducido significativamente en la última década, a pesar de los esfuerzos para lograr la cobertura sanitaria universal a través de la reforma del SP.",3,ejempl nãºmer consult extern especial cad mil derechohabient sp cad mil derechohabient imss issste respect si bien diferent pued reflej neces desigual liger mayor issste pued justific maner nãºmer recet pud ser surt total deb falt existent dentr sp dentr imss segãºn dat encuest aunqu propi cifr institut ss sugier tas altas recet surt ambas cifr encuentr altas ocde gast bolsill reduc signific ultim dec pes esfuerz logr cobertur sanitari universal traves reform sp
1,"En 2007, el gobierno central financió directamente solo el 0,3% del gasto total en salud, pero realizó transferencias específicas para el gasto en salud que ascendieron a otro 5,6% del gasto total. La mayor parte del gasto presupuestario en salud lo realizan los gobiernos de los condados. Por lo tanto, los condados pobres solo pueden ofrecer un bajo nivel de atención a la población local. El gobierno provincial proporciona subsidios del lado de la oferta a los hospitales.",3,gobiern central financ direct sol gast total salud realiz transferent especif gast salud ascend gast total mayor part gast presupuestari salud realiz gobi cond cond pobr sol pued ofrec baj nivel atencion poblacion local gobiern provincial proporcion subsidi lad ofert hospital
2,"Claramente, hay muchos otros factores en juego, en particular, una variedad de barreras y determinantes sociales. Estos pueden estar relacionados con el género, la clase, la etnia, la casta, el idioma y la religión, y surgen de estructuras, normas y procesos sociales arraigados que aceptan e incluso fomentan distribuciones injustas de la riqueza y los recursos sociales. Por ejemplo, las personas que viven con el VIH y algunas otras enfermedades crónicas enfrentan altos niveles de estigma y discriminación que dificultan su acceso a pruebas médicas, tratamiento, atención y apoyo. Por lo tanto, los gobiernos que buscan mejorar los estándares de salud querrán mejorar la condición de la mujer, brindar servicios de salud reproductiva efectivos, implementar programas integrales de lactancia materna en línea con las recomendaciones actuales y aumentar las presiones comunitarias contra el aborto selectivo por sexo. Las personas pobres, que carecen de educación e información, o de fondos para opciones más saludables, son más vulnerables al impacto de las ENT que los ricos y es probable que mueran antes. Directa e indirectamente, las ENT tendrán impactos de gran alcance en el progreso hacia los ODM.",3,clar factor jueg particul varied barrer determin social pued relacion gener clas etni cast idiom religion surg estructur norm proces social arraig acept inclus foment distribu injust riquez recurs social ejempl person viv vih enfermedad cronic enfrent altos nivel estigm discrimin dificult acces prueb medic tratamient atencion apoy gobi busc mejor estandar salud querran mejor condicion muj brind servici salud reproduct efect implement program integral lactanci matern line recomend actual aument presion comunitari abort select sex person pobr carec educ inform fond opci

Ahora bien, los modelos de machine learning trabajan con datos numéricos y no con texto crudo. En este sentido, es necesario convertir las frases en un formato numérico.

Este proceso conocido como vectorización se realizará mediante la técnica TF-IDF. Lo anterior, es una medida numérica que expresa cuán relevante es una palabra para un documento en una colección. Este método pondera las palabras, dando menos importancia a las que aparecen frecuentemente en el conjunto de datos y más a las que son únicas en los documentos individuales.



In [ ]:
#Crear una instancia de TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

#Ajustar el modelo al texto limpio y transformarlo en una matriz de características
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Textos_limpio'])

tfidf_matrix es una matriz que contienen los valores TF-IDF de cada palabra.

#**Modelado y evaluación**

A continuación, se aplican tres algoritmos de clasificación diferentes para predecir y relacionar de forma automática la categoría ODS a la que pertenece una opinión basándose en sus características.

La clasificación es ideal para estos casos porque se centra en asignar categorías (ODS) a partir de los datos de entrada, utilizando algoritmos que pueden aprender de los datos etiquetados para hacer predicciones sobre datos no etiquetados.

En primera instancia, se dividirá el conjunto de datos en un conjunto de entreamiento y otro de prueba. Lo anterior, es esencial para evaluar la capacidad del modelo para generalizar nuevos datos que no conoce durante el entrenamiento.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data['sdg'],test_size=0.2,random_state=42)

##Árboles de decisión

**Realizado por Daniela Castrillón**

Teniendo en cuenta el enunciado y los objetivos del Fondo de Poblaciones de las Naciones Unidas en analizar las opiniones de los ciudadanos, un modelo basado en árboles de decisión podría ser una muy buena opción por distintas razones.

**Interpretabilidad**

Los árboles de decisión son fáciles de entender y visualizar, lo que puede ser crucial cuando se necesita explicar las decisiones del modelo a personas no técnicas.

**Manejo de datos no lineales**

Los árboles de decisión pueden modelar relaciones no lineales entre las caracterísrticas sin necesidad de transformaciones o procesamientos de datos complejos, lo cual es adecuado para datos de texto donde la relación entre palabras y significados puede ser altamente no lineal.

**Capacidad para manejar datos numéricos y categóricos**

Los árboles de decisión maneja correctamente tanto las características numéricas como las categóricas.

**Utilidad en la selección de características**

Los árboles de decisión ayudan en la identificación de las características más significativas para la clasificación. Esto puede ser relevante para entender qué palabras o frases son más influyentes al relacionar las opiniones con los ODS específicos, siendo el objetivo clave del proyecto.

In [ ]:
#Inicializar el modelo de árbol de decisión
dt_model = DecisionTreeClassifier(random_state=42)

#Entrenar el modelo
dt_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

Se creó una instancia de DecisionTreeClassifier junto con la reproducibilidad de los resultados, con el fin de asegurar que el modelo construya el árbol de decisión de la misma manera cada vez que se ejecute el código.

Luego, se entrena el modelo para que el algoritmo decida de manera iterativa qué características (variables) aportan la mayor cantidad de información para dividir los datos en grupos homogéneos.

Durante el entrenamiento, el modelo realiza los siguientes pasos:

* Selecciona la mejor característica para dividir el conjunto de entrenamiento en subconjuntos con respecto a la etiqueta objetivo.

* Una vez seleccionada la mejor característica, se crea de un nodo que divide el conjunto de datos según los valores de dicha característica.

* El proceso se repite recursivamente para cada subconjunto resultante hasta que cumpla con ciertos criterios de parada.


In [ ]:
#Evaluar el modelo
y_pred = dt_model.predict(X_test)

Después de entrenar, se evaluó el modelo utilizando el conjunto de prueba para ver que tan bien generaliza los nuevos datos.

Finalmente, se presentan las métricas de evaluación del modelo para medir el desempeño y rendimiento del mismo.

In [ ]:
#Calcular y mostrar las métricas de rendimiento
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nMatriz de Confusión\n", confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación\n", classification_report(y_test, y_pred))

Accuracy: 0.9135802469135802

Matriz de Confusión
 [[228  11  11]
 [ 11 246  11]
 [ 13  13 266]]

Reporte de Clasificación
               precision    recall  f1-score   support

           3       0.90      0.91      0.91       250
           4       0.91      0.92      0.91       268
           5       0.92      0.91      0.92       292

    accuracy                           0.91       810
   macro avg       0.91      0.91      0.91       810
weighted avg       0.91      0.91      0.91       810



Como se evidencia anteriormente, el árbol de decisión está realizando un trabajo notablemente bueno al clasificar los ODS basados en los datos de texto.

**Accuracy**

Esta métrica nos indica que el 91.36% de las veces, el modelo hizo la predicción correctamente, siendo una buena tasa de acierto.

**Matriz de Confusión**

La matriz de confusión muestra cómo se distribuye las predicciones en comparación con las etiquetes verdaderas. Cada fila de la matriz representa las instancias en una clase reeal, mientras que cada columna representa las instanciase en una clase predicha.

* De las 250 instancias del ODS 3, el modelo predijo 228 instancias del ODS 3, pero confundió 11 con ODS 4 y 5.

* De las 268 instancias del ODS 4, el modelo predijo 246 instancias del ODS 4, pero confundió 11 con ODS 3 y 5.

* De las 292 instancias del ODS 5, el modelo predijo 266 instancias del ODS 5, pero confundió 13 con ODS 3 y 4.

**Reporte de Clasificación**

Finalmente, el reporte de clasificación indica la precisión, recall y F1-score del modelo.

* La precisión indica la capacidad del clasificador de no etiquetar como positivo un ejemplo que es negativo. Para el caso de la ODS 3, es correcto el 90% de las veces; para la ODS 4, es correcto el 91% de las veces y; para la ODS 5, es correcto el 92% de las veces.

* El recall indica la capacidad del modelo de encontrar todos los ejemeplos positivos. Para el caso de la ODS 3 y 5 reales, el modelo los encontró el 91% de las veces; para el caso de la ODS 4 reales, el modeo los encontró el 92% de las veces.

* El F1-score es el promedio de la precisión y el recall. Un F1 alto indica un equilibrio entre precisión y recall. Para el caso de la ODS 3 y 4, se obtuvo 91%; para la ODS 5, se obtuvo 92%.

En conclusión, el modelo muestra un buen ajuste y realiza predicciones precisas y confiables.

##Support Vector Machines (SVM)

**Realizado por Juan Ramírez**

Dado el objetivo analizar las opiniones de los ciudadanos, un modelo usando el algoritmo Support Vector Machines (SVM) es una opción robusta para clasificar los comentarios en diferentes categorías relacionadas con los Objetivos de Desarrollo Sostenible (ODS).

**Capacidad para manejar datos de alta dimensionalidad**

SVM es muy eficaz cuando se trata de conjuntos de datos con muchas características, como es el caso de los datos de texto después de la vectorización. Esto lo hace adecuado para la tarea en cuestión, donde cada palabra puede considerarse como una característica.

**Margen máximo y generalización**

El objetivo principal de SVM es encontrar el hiperplano que maximiza el margen entre las clases. Esto tiende a proporcionar una buena capacidad de generalización, lo que es esencial para predecir correctamente en datos no vistos.

In [ ]:
#Inicializar el modelo
svm_model = SVC(kernel='linear', probability=True, random_state=42)

In [ ]:
#Entrenar el modelo
svm_model.fit(X_train, y_train)

SVC(kernel='linear', probability=True, random_state=42)

A continuación, se crea una instancia de SVC, que es una implementación del clasificador SVM en Scikit-learn.

Durante el entrenamiento del modelo el algoritmo realiza varios pasos clave para encontrar el hiperplano óptimo que separa las categorías en los datos:

* El algoritmo comienza con un hiperplano inicial que separa las clases de manera aleatoria o basada en algún criterio simple. Este hiperplano puede no ser óptimo en absoluto; simplemente sirve como punto de partida.

* Posteriormente el algoritmo busca optimizar el hiperplano realizando los siguientes pasos:

 * Identifica los vectores de soporte: Estos son los puntos de datos más cercanos al hiperplano en cada clase.

 * Maximiza el margen: SVM ajusta el hiperplano para maximizar el margen, es decir, la distancia entre los vectores de soporte de diferentes clases y el hiperplano. Un margen mayor generalmente significa mejor generalización del modelo.

 * Minimiza el error de clasificación: Aunque se busca maximizar el margen, el SVM también tiene en cuenta los errores de clasificación, especialmente si las clases no son perfectamente separables.

* El anterior paso de optimizacion se repite hasta encontrar un punto de convergencia donde el algoritmo termina su ejecución.




In [ ]:
#Evaluar el modelo
y_pred = svm_model.predict(X_test)

In [ ]:
#Metricas de rendimiento
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nMatriz de Confusión\n", confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación\n", classification_report(y_test, y_pred))

Accuracy: 0.9765432098765432

Matriz de Confusión
 [[247   2   1]
 [  3 259   6]
 [  0   7 285]]

Reporte de Clasificación
               precision    recall  f1-score   support

           3       0.99      0.99      0.99       250
           4       0.97      0.97      0.97       268
           5       0.98      0.98      0.98       292

    accuracy                           0.98       810
   macro avg       0.98      0.98      0.98       810
weighted avg       0.98      0.98      0.98       810



 **Accuracy**

 Se evidencia un valor de 97,65% de accuracy, Este alto valor de precisión sugiere que el modelo tiene un excelente rendimiento general y es muy efectivo para esta tarea de clasificación multiclase.

 **Matriz de Confusión**

 * De las 250 instancias del ODS 3, el modelo clasificó correctamente 247, mientras que confundió 2 con el ODS 4 y 1 con el ODS 5.

 * De las 268 instancias del ODS 4, 261 fueron clasificadas correctamente, con 3 instancias clasificadas erróneamente como ODS 3 y 6 como ODS 5.

 * De las 292 instancias del ODS 5, 285 fueron clasificadas correctamente, con 7 clasificadas incorrectamente como ODS 4.

 La mayoría de los errores de clasificación son mínimos, con solo unas pocas instancias confundidas entre los ODS. Esto indica que el modelo tiene una capacidad de discriminación fuerte entre las categorías.

**Reporte de Clasificación**

* La precision indica qué tan preciso es el modelo al etiquetar una instancia como perteneciente a una clase determinada, para los ODS 3, 4 y 5 sus valores son 99%, 97% y 98% respectivamente.

* Recall mide la capacidad del modelo para identificar correctamente todas las instancias de una clase, para los ODS 3, 4 y 5 sus valores son 99%, 97% y 98% respectivamente.

* F1-Score es el promedio armónico entre precision y recall. Un F1-Score alto indica un equilibrio entre precisión y recall, lo cual es deseable, para los ODS 3, 4 y 5 sus valores son 99%, 97% y 98% respectivamente.

El modelo SVM demuestra un rendimiento alto, con una precisión general del 97.65% y altas puntuaciones en las métricas de precision, recall, y F1-Score para todas las clases. Esto sugiere que el modelo es altamente efectivo para la tarea de clasificación de comentarios en las diferentes categorías, logrando un equilibrio sólido entre la capacidad de identificar correctamente las clases y evitar errores de clasificación.




## Naive Bayes
**Elaborado por Mateo Calderón**

Este es un algortimo que se ajusta muy bien a nuestro proyecto, ya que es comunmente usado en casos en donde se requieren sistemas de clasificación de texto.

**Relación con el Teorema de Bayes**

Este clasificador probabilístico se basa en dicho teorema, lo que significa que utiliza probabilidades condicionales y probabilidades previas para calcular las probabilidades posteriores.

**Funcionamiento**

El algoritmo utiliza un supuesto de independencia spone que los predictores en un modelo de este tipo son condicionalmente independientes, es decir, solo se requerirá una única probabilidad para cada variable, lo que facilita el cálculo del modelo. Y aunque el supuesto en el que se basa puede verse como poco realista, funciona bien para tareas de clasificación como esta.


Antes de implementar el algortimo vamos a ver el tamaño de las variables que vamos a utilizar en el modelo.

In [ ]:
print("Tamaño de X_train:", X_train.shape)
print("Tamaño de Y_train:", y_train.shape)
print("Tamaño de X_test (prueba):", X_test.shape)
print("Tamaño de Y_test:", y_test.shape)


Tamaño de X_train: (3239, 9369)
Tamaño de Y_train: (3239,)
Tamaño de X_test (prueba): (810, 9369)
Tamaño de Y_test: (810,)


Como lo vimos en la etapa de limpieza y preparación de los datos, se utilizaron distintos etapas para dejar los datos listos para que el algoritmo pueda trabajar de la mejor forma

En el algoritmo se va a implementar el clasificador Multinomial de Naive Bayes que es muy útil para tareas de clasificación. Usualmente en el mismo algortimo se suelen usar técnicas de vectorización de los datos como el TfidfVectorizer, pero en este caso los datos ingresados como párametro a la función ya lo están.

In [ ]:
def transform_model_data_w_tfidf_vectorizer(X_train, y_train, X_test, y_test):

    #Definicion del modelo de Naive Bayes
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train, y_train)

    #Evaluacion del modelo
    predictions = model.predict(X_test)

    return predictions

In [ ]:
predictions = transform_model_data_w_tfidf_vectorizer(X_train, y_train, X_test, y_test)

print("Accuracy:", accuracy_score(y_test, predictions),"\n")
print("Matriz de Confusión:\n", confusion_matrix(y_test, predictions),"\n")
print("Reporte de Clasificación:\n", classification_report(y_test, predictions))

Accuracy: 0.9679012345679012 

Matriz de Confusión:
 [[241   3   6]
 [  2 261   5]
 [  1   9 282]] 

Reporte de Clasificación:
               precision    recall  f1-score   support

           3       0.99      0.96      0.98       250
           4       0.96      0.97      0.96       268
           5       0.96      0.97      0.96       292

    accuracy                           0.97       810
   macro avg       0.97      0.97      0.97       810
weighted avg       0.97      0.97      0.97       810





**Accuracy**

* El resultado que nos da el modelo es de un accuracy del 96,79%. Lo cuál nos muestra que el algoritmo hizo un muy buen trabajo y fue bastante efectivo para poder clasificar los textos.


**Matriz de Confusión**
* La matriz nos muestra que el modelo obtuvo estos resultados:
 * Clase 3: de los 250 registros, 241 fueron clasificadas correctamente para esa clase, 3 fueron clasificados incorrectamente en la clase 4 y 6 fueron erroneamente asignados a las clase 5.
 * Clase 4: de los 268 registros, 261 fueron correctamente clasificados en esa clase, 2 instancias fueron clasificadas en la clase 3 y 5 incorrectamente en la clase 5.
 * Clase 5: de las 292 instancias de esta clase fueron erroneamente clasificados 1 registro en la clase 3 y 9 en la clase 4, mientras que 282 fueron correctamente clasificados

**Reporte de Clasificación**

* Estos son los resultados que nos da este reporte:

 * La precision que muestra qué tan preciso es el modelo en su tarea de clasificar nos muestra los siguientes resultados, ODS 3: 99%, ODS 4: 96% y ODS 5: 96%.

 * El Recall es el que mide la capacidad del modelo para identificar correctamente todas las instancias de una clase, estos fueron los resultados, ODS 3: 96%, ODS 4: 97% y ODS 5: 97%.

 * F1-Score es el promedio armónico entre precision y recall. El F1-Score esta directamente relacionado con la calidad del modelo, en esta etapa los resultados fueron, ODS 3: 98%, ODS 4: 96% y ODS 5: 96%.


#**Conclusiones**

Teniendo en cuenta los resultados de los diferentes algoritmos, se concluye que se hará uso del algoritmo Support Vector Machines (SVM) ya que fue el modelo con mayor precisión con un 97.65%

Este algoritmo ha demostrado ser el que mejor se adapta a las necesidades del negocio de clasificar las opiniones de los ciudadanos en una ODS específica.

Finalmente, se encontraran insights valiosos para el cliente con respecto a este algoritmo.

In [ ]:
# Obtener los coeficientes del modelo
coef = svm_model.coef_.toarray()

# Crear un DataFrame con las características y sus coeficientes
feature_names = tfidf_vectorizer.get_feature_names_out()
coef_df = pd.DataFrame(coef, columns=feature_names)

for i in range(coef_df.shape[0]):  # Iterar sobre cada clase
    print(f"\nPalabras clave más importantes para el ODS {3+i}:")
    class_coef = coef_df.iloc[i, :]  # Coeficientes para la clase `i`
    important_features_idx = np.argsort(class_coef)[::-1]  # Ordenar por importancia descendente
    top_words = [feature_names[idx] for idx in important_features_idx[:30]]

    # Mostrar las palabras clave y sus coeficientes
    for word in top_words:
        print(f"{word}: {class_coef[word]}")





Palabras clave más importantes para el ODS 3:
salud: 3.8195012063210285
sanitari: 1.9950875092364007
atencion: 1.704463297807135
mental: 1.6731461080470331
pacient: 1.6617033127124583
medic: 1.4893462073962187
hospital: 1.3166687199563527
enfermedad: 1.2693263701646371
enfermed: 1.1611015450363076
mortal: 1.100782478451598
tratamient: 1.0652857386877674
control: 1.0387704381916278
drog: 0.986169148083376
esalud: 0.954043713459996
hospitalari: 0.9397159949400622
medicament: 0.9261564655796224
consum: 0.9025703589165799
muert: 0.897679919761197
directric: 0.8869978662615305
suicidi: 0.8673231602364003
esper: 0.8668770098801807
clinic: 0.8643814048514342
cme: 0.859273604420612
diabet: 0.8182367937441003
abus: 0.8168174900908555
health: 0.8122860632843418
vid: 0.762252690409667
cobertur: 0.7566280446914145
trastorn: 0.7445972378086436
farmaceut: 0.7299165400997464

Palabras clave más importantes para el ODS 4:
salud: 3.039758545529626
sanitari: 1.843584690873858
drog: 1.7466592425188887
m